_한국어로 기계번역됨_


# 서브그래프의 입력 및 출력을 변환하는 방법

서브그래프 상태가 부모 그래프 상태와 완전히 독립적일 수 있습니다. 즉, 두 그래프 사이에 겹치는 채널(키)이 없을 수 있습니다. 예를 들어, 여러 ReAct 에이전트의 도움으로 보고서를 작성해야 하는 감독 에이전트가 있을 수 있습니다. ReAct 에이전트 서브그래프는 메시지 목록을 추적할 수 있지만 감독 에이전트는 사용자 입력과 최종 보고서만 필요하고 메시지를 추적할 필요는 없습니다.

이러한 경우, 서브그래프를 호출하기 전에 입력을 변환하고 반환하기 전에 출력을 변환해야 합니다. 이 가이드는 그 방법을 보여줍니다.

## 설정

먼저 필요한 패키지를 설치해 보겠습니다.


In [1]:
%%capture --no-stderr
%pip install -U langgraph


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 빠르게 발견하고 성능을 향상시키십시오. LangSmith를 사용하면 LangGraph로 구축한 LLM 앱을 디버깅, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대해 더 알아보려면 <a href="https://docs.smith.langchain.com">여기</a>를 읽어보십시오.
    </p>
</div>


## 그래프와 서브그래프 정의하기

3개의 그래프를 정의해 봅시다:
- 부모 그래프
- 부모 그래프에 의해 호출되는 자식 서브그래프
- 자식 그래프에 의해 호출되는 손자 서브그래프


손자녀를 정의하다


In [1]:
from typing_extensions import TypedDict
from langgraph.graph.state import StateGraph, START, END


class GrandChildState(TypedDict):
    my_grandchild_key: str


def grandchild_1(state: GrandChildState) -> GrandChildState:
    # NOTE: child or parent keys will not be accessible here
    return {"my_grandchild_key": state["my_grandchild_key"] + ", how are you"}


grandchild = StateGraph(GrandChildState)
grandchild.add_node("grandchild_1", grandchild_1)

grandchild.add_edge(START, "grandchild_1")
grandchild.add_edge("grandchild_1", END)

grandchild_graph = grandchild.compile()


In [2]:
grandchild_graph.invoke({"my_grandchild_key": "hi Bob"})


{'my_grandchild_key': 'hi Bob, how are you'}

### 어린이를 정의하다


In [3]:
class ChildState(TypedDict):
    my_child_key: str


def call_grandchild_graph(state: ChildState) -> ChildState:
    # NOTE: parent or grandchild keys won't be accessible here
    # we're transforming the state from the child state channels (`my_child_key`)
    # to the child state channels (`my_grandchild_key`)
    grandchild_graph_input = {"my_grandchild_key": state["my_child_key"]}
    # we're transforming the state from the grandchild state channels (`my_grandchild_key`)
    # back to the child state channels (`my_child_key`)
    grandchild_graph_output = grandchild_graph.invoke(grandchild_graph_input)
    return {"my_child_key": grandchild_graph_output["my_grandchild_key"] + " today?"}


child = StateGraph(ChildState)
# NOTE: we're passing a function here instead of just compiled graph (`child_graph`)
child.add_node("child_1", call_grandchild_graph)
child.add_edge(START, "child_1")
child.add_edge("child_1", END)
child_graph = child.compile()


In [4]:
child_graph.invoke({"my_child_key": "hi Bob"})


{'my_child_key': 'hi Bob, how are you today?'}

<div class="admonition info">
    <p class="admonition-title">참고</p>
    <p>
    우리는 <code>grandchild_graph</code> 호출을 별도의 함수(<code>call_grandchild_graph</code>)로 감싸서 입력 상태를 변환한 후 손주 그래프를 호출하고, 손주 그래프의 출력을 다시 자식 그래프 상태로 변환합니다. 변환 없이 <code>grandchild_graph</code>를 직접 <code>.add_node</code>에 전달하면, 자식과 손주 상태 간에 공유된 상태 채널(키)이 없기 때문에 LangGraph는 오류를 발생시킵니다.
    </p>
</div>


자식 및 손자 서브그래프는 부모 그래프와 공유되지 않는 **독립적인** 상태를 갖고 있음을 유의하세요.


부모를 정의하다


In [5]:
class ParentState(TypedDict):
    my_key: str


def parent_1(state: ParentState) -> ParentState:
    # NOTE: child or grandchild keys won't be accessible here
    return {"my_key": "hi " + state["my_key"]}


def parent_2(state: ParentState) -> ParentState:
    return {"my_key": state["my_key"] + " bye!"}


def call_child_graph(state: ParentState) -> ParentState:
    # we're transforming the state from the parent state channels (`my_key`)
    # to the child state channels (`my_child_key`)
    child_graph_input = {"my_child_key": state["my_key"]}
    # we're transforming the state from the child state channels (`my_child_key`)
    # back to the parent state channels (`my_key`)
    child_graph_output = child_graph.invoke(child_graph_input)
    return {"my_key": child_graph_output["my_child_key"]}


parent = StateGraph(ParentState)
parent.add_node("parent_1", parent_1)
# NOTE: we're passing a function here instead of just a compiled graph (`<code>child_graph</code>`)
parent.add_node("child", call_child_graph)
parent.add_node("parent_2", parent_2)

parent.add_edge(START, "parent_1")
parent.add_edge("parent_1", "child")
parent.add_edge("child", "parent_2")
parent.add_edge("parent_2", END)

parent_graph = parent.compile()


<div class="admonition info">
    <p class="admonition-title">참고</p>
    <p>
    우리는 입력 상태를 변환한 후 자식 그래프를 호출하고 자식 그래프의 출력을 부모 그래프 상태로 다시 변환하는 별도의 함수(<code>call_child_graph</code>)에서 <code>child_graph</code> 호출을 감싸고 있습니다. 변환 없이 <code>child_graph</code>를 <code>.add_node</code>에 직접 전달하면 부모 상태와 자식 상태 간의 공유 상태 채널(키)이 없기 때문에 LangGraph에서 오류가 발생합니다.
    </p>
</div>

부모 그래프를 실행하고 자식 및 손자 서브그래프를 올바르게 호출하는지 확인합시다:


In [6]:
parent_graph.invoke({"my_key": "Bob"})


{'my_key': 'hi Bob, how are you today? bye!'}

완벽해! 부모 그래프는 자식 및 손자 서브그래프를 올바르게 호출하고 있습니다(우리의 원래 "my_key" 상태 값에 ", 어떻게 지내세요"와 "오늘?"이 추가되었기 때문에 알고 있습니다).
